In [1]:
from math import sqrt

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

import keras
from keras import models, layers, callbacks, constraints, backend, activations
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier


from matplotlib import pyplot
import matplotlib.patches as mpatches

import datetime
import pathlib
import time
import json
import sys
import os

%matplotlib inline
pd.options.display.max_columns = None

/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# scale train and test data to [-1, 1]
def scale(train, test):
    # fit scaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)

    # transform train
    train_scaled = scaler.transform(train)
    # transform test
    test_scaled = scaler.transform(test)

    return scaler, train_scaled, test_scaled

# inverse scaling for a forecasted value
def invert_scale(scaler, X, y):
    
    # combine data back into single matrix for scaling
    features_and_labels_scaled = np.append(X, y, axis=1)
    features_and_labels = scaler.inverse_transform(features_and_labels_scaled)
    
    return features_and_labels

In [3]:
# create and ann_model to be used with sklearn wrapper
def ann_model(features = 1,
              output_size = 1,
              batch_size = 1,
              epochs = 1,
              hidden_layers = [{'neurons': 50, 'dropout': 0.1 }],
              activation = 'relu',
              optimizer = 'adam',
              loss = 'categorical_crossentropy',
              noise = (False, 0.07),
              metrics = ['accuracy'],
              verbose = 0,
             ):
    
    
    """
    from keras.layers import Input, Dense
    from keras.models import Model

    # This returns a tensor
    inputs = Input(shape=(784,))

    # a layer instance is callable on a tensor, and returns a tensor
    x = Dense(64, activation='relu')(inputs)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(10, activation='softmax')(x)

    # This creates a model that includes
    # the Input layer and three Dense layers
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    """
              
    # create sequenital model
    
    inputs = layers.Input(shape=(features,))
    
    count = 1
    x = None
    previous = inputs
    
    # makes every hidden layer
    for i in range(len(hidden_layers)):

        layer = hidden_layers[i]
        
        if noise[0]:
            
            x = layers.GaussianNoise(noise[1])(previous)
            previous = x

        x = layers.Dense(
            units = layer['neurons'],
            activation = activation,
            name='ANN_hidden_layer_{}'.format(i + 1)
        )(previous)
        
        x = layers.Dropout(
            rate = layer['dropout']
        )(x)
        previous = x
           
    # makes output layer
    outputs = layers.Dense(
        units = output_size,
        activation = 'softmax',
        name = 'output_layer',
    )(previous)
    
    model = models.Model(
        inputs = inputs, 
        outputs = outputs
    )
      
    model.compile(
        loss = loss,
        optimizer = optimizer,
        metrics = metrics,
    )

    return model

## load datasets

In [4]:
train_dataset = pd.DataFrame.from_csv('../dataset/train.csv', index_col=None)
test_dataset = pd.DataFrame.from_csv('../dataset/test.csv', index_col=None)

/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  if __name__ == '__main__':
/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  from ipykernel import kernelapp as app


## explore dataset

In [5]:
# train_dataset.describe()

In [6]:
# train_dataset.info()

In [7]:
# test_dataset.describe()

In [8]:
# test_dataset.info()

## split training into X_train, y_train & rename train_dataset into X_test

In [9]:
y_train = pd.get_dummies(train_dataset['Cover_Type'],  prefix='Cover_Type_').values
X_train = train_dataset.drop('Cover_Type', axis=1).values

X_train.shape, y_train.shape

((15120, 55), (15120, 7))

In [10]:
X_test = test_dataset.values
X_test.shape

(565892, 55)

### try adaboost

In [11]:
# y = y_train[:, 0]
# y.shape

# # base_clf = 
# adaboost_clf = AdaBoostClassifier(
# #     base_estimator = base_clf, 
#     n_estimators=200)
# scores = cross_val_score(adaboost_clf, X_train, y)
# scores.mean()      

### keras ann test

In [12]:
output_size = y_train.shape[1]
features = X_train.shape[1]
batch_size = 1
epochs = 50
hidden_layers = [{'neurons': 50, 'dropout': 0.1}]
verbose = 2

In [13]:
model_test = ann_model(
    features = features,
    output_size = output_size,
    batch_size = batch_size,
    epochs = epochs,
    hidden_layers = hidden_layers,
#     activation = 'relu',
#     optimizer = 'adam',
#     loss = 'categorical_crossentropy',
    noise = (False, 0.07),
#     metrics = ['acc'],
#     verbose = 0,
)

In [14]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaped = scaler.transform(X_test)

/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [15]:
# model_test.fit(X_train, y_train, verbose = verbose, epochs = epochs)
model_test.fit(X_train_scaled, y_train, verbose = verbose, epochs = epochs)

Epoch 1/50
 - 1s - loss: 1.1538 - acc: 0.5567
Epoch 2/50
 - 1s - loss: 0.8348 - acc: 0.6530
Epoch 3/50
 - 1s - loss: 0.7804 - acc: 0.6741
Epoch 4/50
 - 1s - loss: 0.7420 - acc: 0.6863
Epoch 5/50
 - 1s - loss: 0.7091 - acc: 0.7036
Epoch 6/50
 - 1s - loss: 0.6844 - acc: 0.7150
Epoch 7/50
 - 1s - loss: 0.6677 - acc: 0.7177
Epoch 8/50
 - 1s - loss: 0.6465 - acc: 0.7294
Epoch 9/50
 - 1s - loss: 0.6320 - acc: 0.7331
Epoch 10/50
 - 1s - loss: 0.6187 - acc: 0.7428
Epoch 11/50
 - 1s - loss: 0.6096 - acc: 0.7446
Epoch 12/50
 - 1s - loss: 0.5986 - acc: 0.7507
Epoch 13/50
 - 1s - loss: 0.5924 - acc: 0.7534
Epoch 14/50
 - 1s - loss: 0.5837 - acc: 0.7573
Epoch 15/50
 - 1s - loss: 0.5777 - acc: 0.7573
Epoch 16/50
 - 1s - loss: 0.5701 - acc: 0.7646
Epoch 17/50
 - 1s - loss: 0.5644 - acc: 0.7662
Epoch 18/50
 - 1s - loss: 0.5581 - acc: 0.7667
Epoch 19/50
 - 1s - loss: 0.5537 - acc: 0.7714
Epoch 20/50
 - 1s - loss: 0.5489 - acc: 0.7705
Epoch 21/50
 - 1s - loss: 0.5448 - acc: 0.7697
Epoch 22/50
 - 1s - lo

In [16]:
y_hat = model_test.predict(X_train_scaled)

In [17]:
for i in range(7):
    print('({}, {})'.format(max(y_hat[:,i]), min(y_hat[:,i])))

(0.9998540878295898, 0.0)
(1.0, 1.7715477039630958e-15)
(0.9964455962181091, 0.0)
(0.9996896982192993, 0.0)
(0.997796893119812, 0.0)
(0.9992781281471252, 0.0)
(1.0, 0.0)


array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [21]:
accuracy_score(y_train, np.around(y_hat))

0.8018518518518518

### gridsearch and wrapper

In [21]:
model = KerasClassifier(build_fn = ann_model, features = features, output_size = output_size)

In [22]:
epochs = [100]

l1 = [{'neurons': 50, 'dropout': 0.1}]
l2 = [{'neurons': 100, 'dropout': 0.1}, {'neurons': 50, 'dropout': 0.1}]
l3 = [{'neurons': 100, 'dropout': 0.1}, {'neurons': 50, 'dropout': 0.1}, {'neurons': 25, 'dropout': 0.1}]
hidden_layers = [l1, l2, l3]

noises = [(False, 0.07)]

param_grid = dict(       
                  epochs = epochs,
                  hidden_layers = hidden_layers,
#                   activation = activations,
#                   optimizer = optimizers,
#                   loss = losses,
                  noise = noises,
#                   metrics = metrics,
                 )


In [23]:
grid = GridSearchCV(
    estimator = model,
    param_grid = param_grid,
    n_jobs = 1,
)

In [24]:
grid_result = grid.fit(X_train, y_train, verbose = verbose)

Epoch 1/100
 - 1s - loss: 0.0330 - acc: 0.1589
Epoch 2/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 3/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 4/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 5/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 6/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 7/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 8/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 9/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 10/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 11/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 12/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 13/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 14/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 15/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 16/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 17/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 18/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 19/100
 - 1s - loss: 1.1921e-07 - acc: 0.1590
Epoch 20/100
 - 1s - loss

KeyboardInterrupt: 